# Margins

LSDB is able to perform work with datasets larger than memory by partitioning the datasets and working on one partition at a time. When performing some tasks like cross-matching that compare objects that are nearby spatially, for each data point we need to have all the data points nearby loaded at the same time so that we can make those comparisons without missing any matches. We take advantage of HiPSCat's spatial partitioning to ensure this, but there is one case where this isn't true. At the edges of each partition, the data points get cut off, so for something like cross-matching, we wouldn't get accurate results since the algorithm would not have the all the nearby points for any point near the edge if we only ever look at one partition at a time.

![Why we might miss matches without margins](static/margin-boundary.png)
*Here we see an example of a boundary between HEALPix pixels, where the green points are in one partition and the red points in another. Working with one partition at a time, we would miss potential matches with points close to the boundary*

To solve this, we could try to also load the neighboring partitions for each partition we crossmatch. However, this would mean needing to load lots of unnecessary data, slowing down operations and causing issues with running out of memory. So for each catalog we also create a margin cache. This means that for each partition, we create a file that contains the points in the catalog within a certain distance to the pixel's boundary.

![Why we might miss matches without margins](static/margin-pix.png)
*An example of a margin cache (orange) for the same green pixel. The margin cache for this pixel contains the points within 10 arcseconds of the boundary.*

## Loading a Margin Catalog

The margin cache is stored as a separate HiPSCat catalog with the same partitioning as the main catalog.

To load a margin cache, we first load the margin catalog the same as other HiPSCat catalogs with a `lsdb.read_hipscat` call.

In [ ]:
import os.path

import lsdb

surveys_path = "https://epyc.astro.washington.edu/~lincc-frameworks/half_degree_surveys"

In [ ]:
ztf_margin_path = f"{surveys_path}/ztf/ztf_object_margin"
ztf_margin = lsdb.read_hipscat(ztf_margin_path)
ztf_margin

Then we load the object catalog, setting the `margin_cache` parameter to the margin catalog we have just loaded

In [ ]:
ztf_object_path = f"{surveys_path}/ztf/ztf_object"
ztf_object = lsdb.read_hipscat(ztf_object_path, margin_cache=ztf_margin)
ztf_object

The `ztf_object` catalog has now been loaded with its margin cache (This dataset is a small sample of the full DR14 Catalog). We can plot the catalog and its margin to see this.

In [ ]:
# Defining a function to plot the points in a pixel and the pixel boundary

import numpy as np
from matplotlib.patches import Polygon
from matplotlib import pyplot as plt
import healpy as hp


def plot_points(
    pixel_dfs, order, pixel, colors, ra_columns, dec_columns, xlim=None, ylim=None, markers=None, alpha=1
):
    ax = plt.subplot()

    # Plot hp pixel bounds
    nsides = hp.order2nside(order)
    pix0_bounds = hp.vec2dir(hp.boundaries(nsides, pixel, step=100, nest=True), lonlat=True)
    lon = pix0_bounds[0]
    lat = pix0_bounds[1]
    vertices = np.vstack([lon.ravel(), lat.ravel()]).transpose()
    p = Polygon(vertices, closed=True, edgecolor="#3b81db", facecolor="none")
    ax.add_patch(p)

    if markers is None:
        markers = ["+"] * len(pixel_dfs)

    # plot the points
    for pixel_df, color, ra_column, dec_column, marker in zip(
        pixel_dfs, colors, ra_columns, dec_columns, markers
    ):
        ax.scatter(
            pixel_df[ra_column].values,
            pixel_df[dec_column].values,
            c=color,
            marker=marker,
            linewidths=1,
            alpha=alpha,
        )

    # plotting configuration
    VIEW_MARGIN = 2
    xlim_low = np.min(lon) - VIEW_MARGIN if xlim is None else xlim[0]
    xlim_high = np.max(lon) + VIEW_MARGIN if xlim is None else xlim[1]
    ylim_low = np.min(lat) - VIEW_MARGIN if ylim is None else ylim[0]
    ylim_high = np.max(lat) + VIEW_MARGIN if ylim is None else ylim[1]

    plt.xlim(xlim_low, xlim_high)
    plt.ylim(ylim_low, ylim_high)
    plt.xlabel("ra")
    plt.ylabel("dec")
    plt.show()

In [ ]:
# the healpix pixel to plot
order = 3
pixel = 434

# Plot the points from the specified ztf pixel in green, and from the pixel's margin cache in red
plot_points(
    [
        ztf_object.get_partition(order, pixel).compute(),
        ztf_object.margin.get_partition(order, pixel).compute(),
    ],
    order,
    pixel,
    ["green", "red"],
    ["ra", "ra"],
    ["dec", "dec"],
    xlim=[179.5, 180.1],
    ylim=[9.4, 10.0],
)

## Using the Margin Catalog

Performing operations like cross-matching and joining require a margin to be loaded in the catalog on the right side of the operation. If this right catalog has been loaded with a margin, the function will be carried out accurately using the margin, and by default will throw an error if the margin has not been set. This can be overwritten using the `require_right_margin` parameter, but this may cause inaccurate results!

We can see this trying to perform a crossmatch with gaia

In [ ]:
gaia = lsdb.read_hipscat(f"{surveys_path}/gaia")
gaia

If we perform a crossmatch with gaia on the left and the ztf catalog we loaded with a margin on the right, the function works and we get the result

In [ ]:
gaia.crossmatch(ztf_object)

If we try the other way around, we have not loaded the right catalog (gaia) with a margin cache, and so we get an error

In [ ]:
ztf_object.crossmatch(gaia)

We can plot the result of the crossmatch below, with the gaia objects in green and the ztf objects in red

In [ ]:
crossmatch_result = gaia.crossmatch(ztf_object)

plot_points(
    [
        crossmatch_result.get_partition(order, pixel).compute(),
        crossmatch_result.get_partition(order, pixel).compute(),
    ],
    order,
    pixel,
    ["green", "red"],
    ["ra_gaia_halfdegree", "ra_ztf_object_halfdegree"],
    ["dec_gaia_halfdegree", "dec_ztf_object_halfdegree"],
    xlim=[179.5, 180.1],
    ylim=[9.4, 10.0],
    markers=["+", "x"],
    alpha=0.8,
)

## Filtering Catalogs

Any filtering operations applied to the catalog are also performed to the margin.

In [ ]:
small_sky_box_filter = ztf_object.box(ra=[179.9, 180], dec=[9.5, 9.7])

# Plot the points from the filtered ztf pixel in green, and from the pixel's margin cache in red
plot_points(
    [
        small_sky_box_filter.get_partition(order, pixel).compute(),
        small_sky_box_filter.margin.get_partition(order, pixel).compute(),
    ],
    order,
    pixel,
    ["green", "red"],
    ["ra", "ra"],
    ["dec", "dec"],
    xlim=[179.5, 180.1],
    ylim=[9.4, 10.0],
)

## Avoiding Duplicates

Joining the margin cache to the catalog's data would introduce duplicate points, where points near the boundary would appear in the margin of one partition, and the main file of another partition. To avoid this, we keep two separate task graphs, one for the catalog and one for its margin. For operations that don't require the margin, the task graphs are kept separate, and when `compute` is called on the catalog, only the catalog's task graph is computed without joining the margin or even loading the margin files. For operations like crossmatching that require the margin, the task graphs are combined with the margin joined and used. For these operations, we use only the margin for the right side of the catalog. This keeps the results accurate, but since there are no duplicates of the left catalog points, there are no duplicate results.